In [26]:
# Import packages

import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
from pathlib import Path

# Important nltk and sentimentanalyzer
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Used for API call
import requests
import json
import datetime
from datetime import date, timedelta


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Ryan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [27]:
# Create API key variable
api_key=os.getenv("CRYPTONEWS_API_KEY")
print(type(api_key))



<class 'str'>


In [28]:
# Instantiate sentiment analyzer
analyzer=SentimentIntensityAnalyzer()

In [51]:
# Datetime format for while loop to call API
start_date = datetime.date(2021, 4, 1)
end_date = datetime.date(2021, 9, 1)
delta = datetime.timedelta(days=1)

# Create list to store data
solana_sentiment=[]

# Runs while loop to pull daily articles from API                         
while start_date <= end_date:
        
        # Converts start_date to string
        start=start_date.strftime("%m%d%Y")
        
        # Counter for start date
        start_date+=delta         
        
        #Calls API
        solana_articles=f"https://cryptonews-api.com/api/v1?tickers=SOL&items=50&date={start}-{start}&token={api_key}"
        
        # Formats API into JSON
        response_data=requests.get(solana_articles)
        response_content=response_data.content
        data=response_data.json()
        
        # For loop to get sentiment on each article 
        for article in data["data"]:
            try:
                date=article["date"]
                text=article["text"]
                sentiment=analyzer.polarity_scores(text)
                compound=sentiment["compound"]
                pos=sentiment["pos"]
                neu=sentiment["neu"]
                neg=sentiment["neg"]
                
                # Creates binary variable based on positive score
                if pos >= .1:
                    pos_column = 1
                else: 
                    pos_column=0
                # Appends sentiment analysis to solana list    
                solana_sentiment.append({
                    "date":date,
                    "text":text,
                    "compound":compound,
                    "positive":pos,
                    "negative":neg,
                    "neutral":neu,
                    "binary score":pos_column
                }) 
                
            except AttributeError:
                pass

In [52]:
# Creates solana dataframe from list
solana_df=pd.DataFrame(solana_sentiment)

# Add columns to dataframe
columns = ["date","text","compound","positive","negative","neutral","binary score"]    

# Adds column names to dataframe
solana_df=solana_df[columns]

# Formats date column
solana_df['date'] = pd.to_datetime(solana_df['date']).dt.date


# Sorts values by date and drops duplicates
solana_df=solana_df.sort_values(by=["date"])
solana_df.drop_duplicates

solana_df

date                                               text  compound  \
0    2021-04-06  Prominent crypto trader and analyst Cantering ...    0.8402   
1    2021-04-06  Ethereum (ETH) is re-testing the previous all-...    0.4767   
2    2021-04-07  The Solana price had a significant rally in th...    0.9382   
3    2021-04-09  Solana (SOL) has been increasing rapidly since...   -0.3182   
4    2021-04-10  A widely-followed crypto strategist and trader...    0.8074   
..          ...                                                ...       ...   
568  2021-09-01  Blockchain analytics firm Santiment is pointin...   -0.5267   
567  2021-09-01  With August coming to an end, many in the cryp...    0.0498   
577  2021-09-01  Defying the slump in the broader crypto market...   -0.4215   
571  2021-09-01  Cardano ADA/USD is a proof-of-stake (PoS) publ...    0.7906   
578  2021-09-01  It has been a dramatic bull run for Solana inv...    0.0000   

     positive  negative  neutral  binary score  
0       0.136     0.000    0.864             1  
1       0.170     0.000    0.830             1  
2       0.280     0.000    0.720             1  
3       0.080     0.089    0.831             0  
4       0.128     0.000    0.872             1  
..        ...       ...      ...           ...  
568     0.000     0.053    0.947             0  
567     0.111     0.126    0.762             1  
577     0.000     0.052    0.948             0  
571     0.108     0.000    0.892             1  
578     0.000     0.000    1.000             0  

[579 rows x 7 columns]

In [55]:
# Creates mean score per day
solana_df=solana_df.groupby(by='date').mean()

In [56]:
path=Path("Data/solana_sentiment_df.csv")
solana_df.to_csv(path,index="date")